In [1]:
import requests
import math
import numpy as np
from bs4 import BeautifulSoup as bs

In [2]:
elo_ratings_url = "http://elofootball.com/"
elo_ratings_page = requests.get(elo_ratings_url)

In [3]:
if elo_ratings_page:
    page_parsed = bs(elo_ratings_page.text, 'html.parser')
    print("HTML parsed succesfully.")
else:
    print("Could not parse the HTML.")

HTML parsed succesfully.


In [4]:
all_trs = page_parsed.find_all("tr")
print("Total tr tags in the HTML:", len(all_trs))

Total tr tags in the HTML: 2366


In [5]:
teams = {}

# Since we cannot identify the table correctly, let's find the tr tags with the correct amount of td tags
for tr in all_trs:
    all_tds = tr.find_all("td")
    
    if all_tds and len(all_tds) == 18:
        teams[all_tds[1].text.strip()] = int(all_tds[8].text)
        
print("Total teams:", len(teams))

Total teams: 50


In [20]:
knockout_games = [
    [
        "Atlético Madrid",
        "Borussia Dortmund",
    ],
    [
        "Paris Saint-Germain",
        "FC Barcelona",
    ],
    [
        "Arsenal FC",
        "Bayern München",
    ],
    [
        "Real Madrid",
        "Manchester City",
    ],
]

print("Quarter finals teams:\n")

for game in knockout_games:
    for team in game:
        print(f"{team} - {teams[team]}")

Quarter finals teams:

Atlético Madrid - 2155
Borussia Dortmund - 2177
Paris Saint-Germain - 2214
FC Barcelona - 2176
Arsenal FC - 2241
Bayern München - 2234
Real Madrid - 2324
Manchester City - 2398


In [21]:
def get_winning_probability(elo_a, elo_b):
    # ELO difference
    delta_elo = elo_b - elo_a
    
    # Using a logistic function to convert the Team A winning probabilites
    proba_a = 1 / (1 + math.pow(10, delta_elo / 400))
    
    return proba_a

def simulate_game(proba_a):
    if np.random.rand() < proba_a:
        return 'A'  # Team A wins
    else:
        return 'B'  # Team B wins
    
def monte_carlo_simulation(team_a, team_b, num_simulations):
    team_a_wins = 0
    team_b_wins = 0
    
    # Probability based simply on the ELO rating
    elo_proba_a = get_winning_probability(team_a['elo'], team_b['elo'])
    
    for _ in range(num_simulations):
        simulation_result = simulate_game(elo_proba_a)
        if simulation_result == 'A':
            team_a_wins += 1
        else:
            team_b_wins += 1
           
    # Probability based on the simulations
    simulation_proba_a = team_a_wins / num_simulations
    simulation_proba_b = team_b_wins / num_simulations
    
    print(f"\n{team_a['team']} ({team_a['elo']}) won {team_a_wins} of {num_simulations} simulations ({round(simulation_proba_a, 2)}%)")
    print(f"{team_b['team']} ({team_b['elo']}) won {team_b_wins} of {num_simulations} simulations ({round(simulation_proba_b, 2)}%)")
    
    if simulation_proba_a > simulation_proba_b:
        predicted_winner = team_a['team']
    else:
        predicted_winner = team_b['team']
    
    print(f"RESULT: {predicted_winner} wins.")
    
    return predicted_winner
    
def simulate_stage(games, num_simulations):
    next_stage = []
    
    for game in games:
        winner = monte_carlo_simulation(
            {
                'team': game[0],
                'elo': teams[game[0]]
            }, 
            {
                'team': game[1],
                'elo': teams[game[1]]
            },
            num_simulations
        )
        
        if next_stage and len(next_stage[-1]) < 2:
            next_stage[-1].append(winner)
        else:
            next_stage.append([winner])
            
    return next_stage
        
while len(knockout_games) > 0 and len(knockout_games[0]) > 1:
    if len(knockout_games) == 4:
        print("\nQUARTER FINALS")
    elif len(knockout_games) == 2:
        print("\nSEMI FINALS")
    else:
        print("\nFINAL")
        
    knockout_games = simulate_stage(knockout_games, 101)


QUARTER FINALS

Atlético Madrid (2155) won 46 of 101 simulations (0.46%)
Borussia Dortmund (2177) won 55 of 101 simulations (0.54%)
RESULT: Borussia Dortmund wins.

Paris Saint-Germain (2214) won 51 of 101 simulations (0.5%)
FC Barcelona (2176) won 50 of 101 simulations (0.5%)
RESULT: Paris Saint-Germain wins.

Arsenal FC (2241) won 46 of 101 simulations (0.46%)
Bayern München (2234) won 55 of 101 simulations (0.54%)
RESULT: Bayern München wins.

Real Madrid (2324) won 31 of 101 simulations (0.31%)
Manchester City (2398) won 70 of 101 simulations (0.69%)
RESULT: Manchester City wins.

SEMI FINALS

Borussia Dortmund (2177) won 51 of 101 simulations (0.5%)
Paris Saint-Germain (2214) won 50 of 101 simulations (0.5%)
RESULT: Borussia Dortmund wins.

Bayern München (2234) won 22 of 101 simulations (0.22%)
Manchester City (2398) won 79 of 101 simulations (0.78%)
RESULT: Manchester City wins.

FINAL

Borussia Dortmund (2177) won 22 of 101 simulations (0.22%)
Manchester City (2398) won 79 of 